In [13]:
import numpy as np
import joblib
from sklearn import svm
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
from SliceDatasetBuilder import CustomHipDataset

class SliceSVMClassifier:
    def __init__(self, kernel='linear', C=1.0):
        """
        Initialize the ImageSVMClassifierWithHOG.

        :param kernel: SVM kernel (default: 'linear')
        :param C: Regularization parameter (default: 1.0)
        """
        self.clf = svm.SVC(kernel=kernel, C=C)

    def _extract_hog_features(self, images):
        hog_features = []
        for image in tqdm(images, desc="Extracting HOG Features"):
            image = resize(image, (128, 128))
            # Extract HOG features
            fd = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')

            hog_features.append(fd)
        return np.array(hog_features)

    def train(self, custom_dataset, test_size=0.3, random_state=None):
        """
        Train the SVM classifier with HOG features using a CustomHipDataset.

        :param custom_dataset: CustomHipDataset object
        :param test_size: Fraction of data to use for testing (default: 0.2)
        :param random_state: Random seed for reproducibility
        """
        X = custom_dataset[0]
        y = custom_dataset[1]

        hog_features = self._extract_hog_features(X)
        X_train, X_test, y_train, y_test = train_test_split(hog_features, y, test_size=test_size, random_state=random_state)

        print(f"Training SVM classifier with {len(X_train)} images and {len(X_test)} test images...")

        
        self.clf.fit(X_train, y_train)

        accuracy = self.clf.score(X_test, y_test)
        return accuracy

    def predict(self, X):
        """
        Make predictions using the trained SVM classifier with HOG features using a CustomHipDataset.

        :param custom_dataset: CustomHipDataset object
        :return: List of predicted labels
        """
        hog_features = self._extract_hog_features(X)
        print(len(hog_features[0]))
        predictions = self.clf.predict(hog_features)
        return predictions
    
    def save_model(self, filename):
        """
        Save the trained SVM classifier model to a file using joblib.

        :param filename: Name of the file to save the model to
        """
        joblib.dump(self.clf, filename)
    
    def load_model(self, filename):
        """
        Load a trained SVM classifier model from a file using joblib.

        :param filename: Name of the file to load the model from
        :return: An instance of SliceSVMClassifier with the loaded model
        """
        self.clf = joblib.load(filename)
        return


In [3]:
# Load the dataset
hips = CustomHipDataset("label_onlyNormal.json", view_types=["axial"])
hips.load_all_dataset()

Loading data/dataset/normalHip\JOR01\axial...
Positive interval of JOR01: [330, 400] for SX
Positive interval of JOR01: [280, 370] for DX
Loading data/dataset/normalHip\JOR02\axial...
Positive interval of JOR02: [250, 380] for SX
Positive interval of JOR02: [340, 445] for DX


Load all dataset: 100%|██████████| 4034/4034 [01:45<00:00, 38.35it/s]


In [4]:
i=1221
print(hips.image_paths[i])
print(hips.labels[i])

data/dataset/normalHip\JOR01\axial\JOR01_290_DX.png
True


In [9]:
# Initialize the SVM classifier

svm_classifier = SliceSVMClassifier()

# Train the SVM classifier with HOG features
accuracy = svm_classifier.train(hips.get_all_dataset())

print(f"Accuracy: {accuracy}")

# Save the trained model to a file
svm_classifier.save_model("axial.pkl")

Extracting HOG Features: 100%|██████████| 4034/4034 [02:59<00:00, 22.45it/s]


Training SVM classifier with 2823 images and 1211 test images...
Accuracy: 1.0


In [14]:
import matplotlib.pyplot as plt

#TEST CLASSIFIER
loaded = SliceSVMClassifier()
# Load the model from a file
loaded.load_model("axial.pkl")

# Load images from a folder
folder_path = "data/dataset/normalHip/JOR09/axial"

image_array = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    image = np.array(Image.open(file_path).convert("RGB").convert("L"))
    image_array.append(image)

# Make predictions using the loaded model
predictions = loaded.predict(image_array)

#print number of 1s
print(f"Number of 1s: {np.count_nonzero(predictions == 1)}")

# Print the predictions
for index, value in enumerate(predictions):
    if value == 1:
        print(f"Value 1 found at index {index+1}")
        plt.imshow(image_array[index], cmap='gray')
        plt.show()



Extracting HOG Features: 100%|██████████| 1468/1468 [01:09<00:00, 20.98it/s]


8100
Number of 1s: 0
